# Query and/or Retrieve catalogs from the AL / JWST Server.

In [ ]:
# Example notebook to query and/or retrieve catalogs from the AL/JWST Server via its API.
#   Utilizes the ALWS_utils.ipynb notebook, which contains API helper classes and methods.
#
#  Written by: Tom Hicks. 5/13/2022.
#  Last Modified: Update for changes in the API.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

%matplotlib inline

In [ ]:
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.wcs import WCS
from astropy.utils.data import download_file
from astropy.visualization import MinMaxInterval, AsinhStretch, SqrtStretch, ImageNormalize
import astropy.units as u

In [ ]:
import pandas as pd
from io import StringIO

#### Load additional "helper" code to help manage interaction with the API:

In [ ]:
%run ALWS_utils.ipynb

#### Create an instance of the API "helper" class which uses the default server base URL (both just loaded).

In [ ]:
api = AlwsApi()

#### Authorize an existing user, storing the resulting JWT access token in the instance of the class:

In [ ]:
## Uncomment the following two lines to see the response when authorization fails:
# resp = api.authorize('jfake@nowhere.com', 'invalid_pwd')
# print(resp.json())
resp = api.authorize('jfake@nowhere.com', 'testydjango2')
# print(resp.json())

## Fetch a catalog file (CSV) from the AL/JWST image server

#### First, let's see what catalogs are available from the server:

In [ ]:
resp = api.get('cats/')
get_results(resp)

#### Alternatively, if we know the name (or filename) of a catalog, we can ask for information on just that catalog. For example, we can ask for information about the **small10** catalog only. For any API query, we can (1) manually construct the desired URL or (2) provide a dictionary of arguments to our "helper" function and allow it to construct the URL. Here is an example of using a manually constructed URL:

In [ ]:
get_results(api.get('cats/by_name/?catalog=small10'))

#### When using the API programmatically, you may often construct the argument dictionary separately and, later, pass it to the helper function. For example:

In [ ]:
# Compute some arguments and then call the 'get' function with those arguments:
#    which_catalog = ...some computation producing a dictionary...
#    ...
#    get_results(api.get('cats/by_name/', params=which_catalog))

#### For simple cases, we can also just construct the argument dictionary manually:

In [ ]:
get_results(api.get('cats/by_name/', params={'catalog': 'small10'}))

#### The catalog **small10** looks like a good test case. Let's fetch the entire catalog and read the resulting CSV into a Pandas DataFrame.
#### _NOTE:_ Use this call carefully, as your computer may not have enough resources to fetch all rows of a large catalog.

In [ ]:
resp = api.get('cats/fetch/', params={'catalog': 'small10'})
csv_string = StringIO(resp.text)
df = pd.read_csv(csv_string)
df

#### Since this is a tiny catalog, we can also get its data as a JSON structure.
#### _NOTE: Use this call carefully, as the JSON response takes more time and memory. (Not recommended for catalogs with more than ~2000 rows)._

In [ ]:
resp = api.get('cats/rows/by_catalog/', params={'catalog': 'small10'})
jcat = get_results(resp)
jcat

#### Each JSON object, corresponding to a row, stores the original table values for that row in a dictionary called 'cols'. We can rearrange the rows into a list of dictionaries, making it easy to read them into a Pandas DataFrame:

In [ ]:
dfj = pd.DataFrame([rec.get('cols') for rec in jcat])
dfj

## Do a Cone Query for matching rows within a catalog:

#### _Note: The AL/JWST respository allows for the storage of heterogenous catalogs whose only common columns are right ascension, declination, and magnitude. Therefore, cone queries must be done per catalog in order to retrieve the proper structure for each catalog._

#### First, let's find out which catalogs contain data rows near a certain coordinate. To do this we use the _query_cone_count_ API call:

In [ ]:
get_results(api.get('cats/query_cone/counts/', params={'dec': -27.831254438, 'ra': 53.200010146, 'sizeArcSec': 30}))

#### Two catalogs appear to contain matching data, with 10 records each. In order to retrieve the matching rows from the **small326** catalog, we need to add the catalog name to the previous dictionary of query arguments:

In [ ]:
resp = api.get('cats/fetch_cone/by_catalog/', params={'dec': -27.831254438, 'ra': 53.200010146, 'sizeArcSec': 30, 'catalog': 'small326'})
# OR
# api.get('cats/fetch_cone/by_catalog/?dec=-27.831254438&ra=53.200010146&sizeArcSec=30&catalog=small326')

#### Since _fetch_cone_ returns its results as CSV, the matching rows are found in the 'text' portion of the response. We can read the matching rows into a Pandas DataFrame, as follows:

In [ ]:
dfq = pd.read_csv(StringIO(resp.text))
dfq

#### _Note: that when data rows are returned from the repository database via a query, the original table column names are automatically prefixed with '**t.**' to distinguish them from other database columns (such as the 'catalog' id column above)._